In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from kmeans import KMeans
from kmeans_printer import print_per_game_kmeans
from pca import print_pca

from symmetric_uncertainty import SUT

from data_common import PERSONALITY_FEATURES, PERSONALITY_FEATURES_NO_LOCUS, EMOTIONS_FEATURES, EMOTIONS_LABELS, DAA, CLUSTER

#paradox = 'ellsberg'
paradox = 'allais'
df = pd.read_csv(f'data/data_{paradox}.csv')

df = df[PERSONALITY_FEATURES + EMOTIONS_FEATURES + [DAA]]

#### Step 1: Find optimal K when clustering by personality traits

In [ ]:
random_state = 18
K_range = range(2, 11)
sils = [KMeans(df, k, features=PERSONALITY_FEATURES, random_state=random_state).silhouette for k in K_range]
plt.plot(K_range, sils, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouettes')
plt.show()

#### Use optimal K (= 2) and plot personality cluster centroids

In [ ]:
K = 2
km = KMeans(df, K, features=PERSONALITY_FEATURES)
for cf, c0, c1 in zip(PERSONALITY_FEATURES, km.centroids[0], km.centroids[1]):
    print(f'{cf:19} {c0:10.3} {c1:10.3}')
print()

print_per_game_kmeans(km, [PERSONALITY_FEATURES_NO_LOCUS], ylim=None, plot_graph=True)

#### Step 2: For each of the clusters we found, perform second level clustering based on emotions

In [ ]:
PERSONALITY_CLUSTER = 'PersonalityCluster'
df[PERSONALITY_CLUSTER] = km.labels
print(df[PERSONALITY_CLUSTER].value_counts())

In [ ]:
print_pca(df, features=km.features, labels=km.labels)

In [ ]:
df[CLUSTER] = df[PERSONALITY_CLUSTER]
print(SUT(df))

In [ ]:
# Use same global properties for KMeans search
K_range = range(2, 11)

#### Emotion-based clusters within personality cluster #0

In [ ]:
df0 = df[df[PERSONALITY_CLUSTER] == 0].copy()
sils = [KMeans(df0, k, features=EMOTIONS_FEATURES, random_state=random_state).silhouette for k in K_range]
print(sils)
plt.plot(K_range, sils, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette')
plt.show()

In [ ]:
km = KMeans(df0, K, features=EMOTIONS_FEATURES, random_state=random_state)
print_per_game_kmeans(km, EMOTIONS_LABELS, plot_graph=True)

In [ ]:
df0[CLUSTER] = km.labels
print(SUT(df0))

#### Emotion-based clusters within personality cluster #1

In [ ]:
df1 = df[df[PERSONALITY_CLUSTER] == 1].copy()
sils = [KMeans(df1, k, features=EMOTIONS_FEATURES, random_state=random_state).silhouette for k in K_range]
print(sils)
plt.plot(K_range, sils, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette')
plt.show()

In [ ]:
km = KMeans(df1, K, features=EMOTIONS_FEATURES, random_state=random_state)
print_per_game_kmeans(km, EMOTIONS_LABELS, plot_graph=True)

In [ ]:
df1[CLUSTER] = km.labels
print(SUT(df1))